In [1]:
from pyimagesearch.transform import four_point_transform
from pyimagesearch import imutils
from skimage.filters import threshold_adaptive
import numpy as np
import argparse
import cv2
import matplotlib.pyplot as plt
from PIL import Image

import theano
import theano.tensor as T
from theano_toolkit import utils as U
from theano_toolkit import hinton
from theano_toolkit import updates
from theano_toolkit.parameters import Parameters

%matplotlib inline



In [ ]:
image = cv2.imread('document-scanner/images/lao.jpg')
#image = cv2.imread('document-scanner/images/braums-4.jpg')
#image = cv2.imread('document-scanner/images/1-original-receipt-view.png')
#ratio = image.shape[0] / 500.0
ratio = image.shape[0] / 650.0
print ("ratio %s" % ratio)
orig = image.copy()
image = imutils.resize(image, height = 650)

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 75, 200)

In [ ]:
plt.imshow(image)
plt.show()
plt.imshow(edged, cmap='gray')
plt.show()

In [ ]:
_, cnts, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:10]
screenCnt = None
i = 0
maxp = 0
selp = 0
selc = None
for c in cnts:
	#print ("countur %d" % i)
	#i = i + 1
	# approximate the contour
	peri = cv2.arcLength(c, True)
	approx = cv2.approxPolyDP(c, 0.02 * peri, True)
	print ("peri is %s" % peri)
	if selp < peri:
		selp = peri
		selc = approx
		screenCnt = approx
	#print ("approx is %s" % approx) 

    
	if len(approx) == 4:
		screenCnt = approx
		break
print("selp %s" % selp)

In [ ]:
cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
#cv2.drawContours(image, cnts, -1, (0, 255, 0), 2)
#cv2.drawContours(image, [selc], 0, (0, 255, 0), 2)

In [ ]:
plt.imshow(image)
plt.show()

In [ ]:
warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
warped = threshold_adaptive(warped, 251, offset = 10)
warped = warped.astype("uint8") * 255

In [ ]:
plt.imshow(imutils.resize(warped, height = 650), cmap='gray')
plt.show()

In [ ]:
fig = plt.figure()
a=fig.add_subplot(1,2,1)
imgplot1 = plt.imshow(imutils.resize(orig, height = 650))
a.set_title('Before')
a=fig.add_subplot(1,2,2)
imgplot2 = plt.imshow(imutils.resize(warped, height = 650), cmap='gray')
a.set_title('After')
plt.show()

In [ ]:
im = Image.fromarray(imutils.resize(warped, height = 650, width = 650))
im.save("final_file.jpeg")


In [ ]:
%%bash
ocropus-nlbin -n final_file.jpeg -o img_out


In [ ]:
%%bash
ocropus-gpageseg -n --maxcolseps 0 --minscale 8.0 img_out/0001.bin.png 

In [ ]:
%%bash
ocropus-rpred -m en-default.pyrnn.gz img_out/0001/*.png

In [ ]:
%%bash
cat img_out/????/??????.txt > ocr.txt